### Compute tf-idf score using MapReduce

In [ ]:
wget https://www.dropbox.com/s/9t4my8o4g3fibwk/tmapper.py
wget https://www.dropbox.com/s/kst1uc3so0sugrz/treducer.py
wget https://www.dropbox.com/s/rojbnquhqsd3oa7/tfmapper.py
wget https://www.dropbox.com/s/xat1ew4l3pxpokr/tfreducer.py
wget https://www.dropbox.com/s/13p3c6hvroef2b9/dmapper.py
wget https://www.dropbox.com/s/s775nolj6ltdq03/dreducer.py
wget https://www.dropbox.com/s/8ahm8fa9m8r0ksw/tfidfmapper.py
chmod +x *.py
    

# compute term occurance
hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-input /user/hadoop/proj/input \
-output /user/hadoop/proj/output \
-file /home/hadoop/tmapper.py \
-mapper /home/hadoop/tmapper.py \
-file /home/hadoop/treducer.py \
-reducer /home/hadoop/treducer.py

#hdfs dfs -ls /user/hadoop/proj/output/
hdfs dfs -rm /user/hadoop/proj/output/_SUCCESS

#hdfs dfs -cat /user/hadoop/proj/output/part-00000

In [ ]:
# compute number of words in each file and combine with the previous output

hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D stream.non.zero.exit.is.failure=false \
-input /user/hadoop/proj/output \
-output /user/hadoop/proj/output1 \
-file /home/hadoop/tfmapper.py \
-mapper /home/hadoop/tfmapper.py \
-file /home/hadoop/tfreducer.py \
-reducer /home/hadoop/tfreducer.py

#hdfs dfs -ls /user/hadoop/proj/output1/
hdfs dfs -rm /user/hadoop/proj/output1/_SUCCESS
#hdfs dfs -cat /user/hadoop/proj/output1/part-00000



In [ ]:
# compute document frequency and add it to the previous output

hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D stream.non.zero.exit.is.failure=false \
-input /user/hadoop/proj/output1 \
-output /user/hadoop/proj/output2 \
-file /home/hadoop/dmapper.py \
-mapper /home/hadoop/dmapper.py \
-file /home/hadoop/dreducer.py \
-reducer /home/hadoop/dreducer.py

#hdfs dfs -ls /user/hadoop/proj/output2/
hdfs dfs -rm /user/hadoop/proj/output2/_SUCCESS
#hdfs dfs -cat /user/hadoop/proj/output2/part-00000


In [ ]:
# compute tf-idf score

hadoop jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D stream.non.zero.exit.is.failure=false \
-input /user/hadoop/proj/output2 \
-output /user/hadoop/proj/output3 \
-file /home/hadoop/tfidfmapper.py \
-mapper /home/hadoop/tfidfmapper.py


#hdfs dfs -ls /user/hadoop/proj/output3/
#hdfs dfs -cat /user/hadoop/proj/output3/part-00000

In [ ]:
#tmapper.py

#!/usr/bin/python
 
import sys
import os
 
for line in sys.stdin:
    # get the file path
    input_fn = os.getenv('mapreduce_map_input_file')
    # only keep the name of file
    name = str(input_fn).split("/")[-1]
    word = str(line.strip())
    word_doc = word + "," + name
    print '%s\t%s' % (word_doc, 1)

In [ ]:
# treducer.py


#!/usr/bin/python
 
import sys
 
current_word = None
current_count = 0
word = None
 
# input comes from STDIN
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
 
    # parse the input we got from mapper.py
    line = line.split('\t')
    word = line[0]
    count = line[1]
 
    # convert count (currently a string) to int
    try:
        count = int(count)
    except ValueError:
        continue
 
    if current_word == word:
        current_count += count
    else:
        if current_word:
            # write result to STDOUT
            print '%s\t%s' % (current_word, current_count)
        current_count = count
        current_word = word


if current_word == word:
    print '%s\t%s' % (current_word, current_count)
    
# output ((word,file),frequency of the word in the file)

In [ ]:
#tfmapper.py

#!/usr/bin/python

# use result of treducer.py, and output (filename, (word,frequency of the word in the file))
import sys
 
for line in sys.stdin:
    line = line.strip()
    kv = line.split('\t')
    wordfilename = kv[0]
    count = kv[1]
    wf = wordfilename.split(',')
    word = wf[0]
    filename = wf[1]
    wc = word + ','+count
    print '%s\t%s' % (filename, wc)

In [ ]:
#tfreducer.py

#!/usr/bin/python
 
import sys
 
current_word = None
prev_filename = None
current_count = 0
word = None
N=0
df={} # this dictionary will store (key = name of file, value = size of that file)
result=[] # this list will store the output of tmapper.py
 
for line in sys.stdin:
    line = line.strip()
    result.append(line)
    filename = line.split("\t")[0]
    wordcount = line.split("\t")[1]
    word = wordcount.split(',')[0]
    count = wordcount.split(',')[1]
    count=int(count)
    if prev_filename == filename:
        N=N+count
    else:
        if prev_filename != None:
            df[prev_filename]=N
        N=count
        prev_filename = filename
df[prev_filename]=N
 
for ele in result:
    filename = ele.split('\t')[0]
    wordcount = ele.split('\t')[1]
    word = wordcount.split(',') [0]
    count = wordcount.split(',') [1]
    for name in df:
        if filename == name:
            wf=word+','+filename
            nN=str(count)+','+str(df[name])            
            print '%s\t%s' % (wf,nN)
            
# output ((word,filename), (frequency of the word, size of that document))

In [ ]:
# dmapper.py

#!/usr/bin/python


# use the output of tfreducer.py
import sys
 
for line in sys.stdin:
    # remove leading and trailing whitespace
    line = line.strip()
    # split the line into words
    wf = line.split('\t')[0]
    nN = line.split('\t')[1]
    w = wf.split(',')[0]
    f = wf.split(',')[1]
    fn=f+','+nN+','+str(1)
    print '%s\t%s' % (w,fn)
    
# output (word,(filename,frequency of the word, size of that document,1))

In [ ]:
# dreducer.py

#!/usr/bin/python
 
import sys
 
prev_word = None
count = 0 # this will be variable for document frequency
 
df={} # will store (key = word, value = (filename,frequency of the word, size of that document,document frequency))
result=[] # will store (word,filename)
# input comes from STDIN
for line in sys.stdin:
    line = line.strip()
    w = line.split('\t')[0]
    fn = line.split('\t')[1]
    f = fn.split(',')[0]
    n = fn.split(',')[1]
    N = fn.split(',')[2]
    c = fn.split(',')[3]
    if prev_word == w:
        count = count+int(c)
    else:
        if prev_word != None:
            q=n+','+N+','+str(count)
            df[prev_word]=q
            j=prev_word+','+f
            result.append(j)
        count=1
        prev_word = w
 
        
q=n+','+N+','+str(count)
df[prev_word]=q
j=prev_word+','+f
result.append(j)
 
for ele in result:
    w = ele.split(',')[0]
    f = ele.split(',')[1]
    for d in df:
        if w == d:
            print '%s\t%s' % (ele,df[d])
# output ((word,filename),(frequency of the word, size of that document,document frequency))

In [ ]:
#tfidfmapper.py

#!/usr/bin/python

# use output of dreducer.py, calculate the score, and output ((word,filename),tf-idf score)
 
import sys
import math
 
D = 5.0
 
for line in sys.stdin:
    line = line.strip()
    wf,nNm=line.split('\t',1)
    wf = line.split('\t')[0]
    nNd = line.split('\t')[1]
    n = nNd.split(',')[0]
    N = nNd.split(',')[1]
    d = nNd.split(',')[2]
    n=float(n)
    N=float(N)
    d=float(d)
    tfidf= (n/N)*math.log(D/d)
    print '%s\t%s' % (wf,tfidf)